# Cleaning the data  
  
In this notebook, I will be loading in the datasets and be cleaning them and pre-processing them for analysis and modeling. This will involve combining datasets from two different sources. One from a Twitter archive and one from the FEC which records all individual campaign contributions for political candidates.  
  
The tweet dataset will have to be cleaned by combining tweets by day. Then the daily tweets just be pre-processed so that it can be more easily analyzed. This includes removing punctuation, getting rid of html, removing emojis and other none-text objects, and lower-casing. the dataset will also have to be count daily favorite counts and total tweets.  

The campaign contribution contains a lot of information that will not be used in this project. I remove this extra information and then index both datasets by day.   

The work flow is as follows:  
  
> - [Import Trump Twitter data](#importing_tweet_data)
> - [Convert Twitter data into datetime](#twitter_date)
> - [Aggregate tweets by day](#day_tweet)
> - [Remove HTML and non-text](#remove)
> - [Import campaign donations](#campaign)
> - [Clean campaign dataset](#camp_clean)
> - [Combine tweet and donation datasets](#combine)
> - [Remove punctuation and lowercase tweets](#remove)
> - [Export clean dataset](#export)
> - [Conclusions](#conclusions)

<a id='importing_tweet_data'></a>
## Import Trump tweets.   

> These are all historical tweets made by Donald Trump between Jan. 1, 2019 and June 30,2019. They do not include any of his re-tweets. Tweets were provided by [Trump Twitter Archive](http://www.trumptwitterarchive.com/archive)

In [2]:
#Load trump tweets
df_tweets= pd.read_csv('../datasets/trump_tweets.csv')

In [3]:
df_tweets.head()

,text,created_at,favorite_count,is_retweet,id_str
0,The leaders of virtually every country that I ...,06-30-2019 00:29:16,148261,False,1145127125512536065
1,The highly respected Farm Journal has just ann...,06-30-2019 00:07:39,101806,False,1145121683193421826
2,I am in South Korea now. President Moon and I ...,06-29-2019 22:35:12,114947,False,1145098417405923330
3,....again with China as our relationship with ...,06-29-2019 22:32:40,84014,False,1145097779632627712
4,....amounts of agricultural product from our g...,06-29-2019 22:32:39,80205,False,1145097776658812929


In [4]:
df_tweets.tail()

,text,created_at,favorite_count,is_retweet,id_str
1855,“General” McChrystal got fired like a dog by O...,01-01-2019 15:32:30,69306,False,1080124615920373760
1856,The Democrats much as I suspected have allocat...,01-01-2019 14:32:01,101836,False,1080109395357380613
1857,Happy New Year!,01-01-2019 14:25:32,277623,False,1080107759755034624
1858,HAPPY NEW YEAR TO EVERYONE INCLUDING THE HATER...,01-01-2019 13:08:29,256114,False,1080088373451206656
1859,Dr. Sebastian Gorka a very good and talented g...,01-01-2019 12:51:34,76622,False,1080084113762197504


In [5]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1860 entries, 0 to 1859
Data columns (total 5 columns):
text              1860 non-null object
created_at        1860 non-null object
favorite_count    1860 non-null int64
is_retweet        1860 non-null bool
id_str            1860 non-null int64
dtypes: bool(1), int64(2), object(2)
memory usage: 60.0+ KB


> A quick glance at the data suggests that it is pretty clean the only thing immediately sticking out is the changing the created_at column to a datetime object and setting it as the index. 
<a id='twitter_date'></a>

In [6]:
df_tweets['Date']= pd.to_datetime(df_tweets['created_at'])

In [7]:
df_tweets.set_index('Date', inplace=True)
df_tweets.sort_index(inplace=True,ascending=True)

In [8]:
df_tweets.head()

,text,created_at,favorite_count,is_retweet,id_str
Date,,,,,
2019-01-01 12:51:34,Dr. Sebastian Gorka a very good and talented g...,01-01-2019 12:51:34,76622,False,1080084113762197504
2019-01-01 13:08:29,HAPPY NEW YEAR TO EVERYONE INCLUDING THE HATER...,01-01-2019 13:08:29,256114,False,1080088373451206656
2019-01-01 14:25:32,Happy New Year!,01-01-2019 14:25:32,277623,False,1080107759755034624
2019-01-01 14:32:01,The Democrats much as I suspected have allocat...,01-01-2019 14:32:01,101836,False,1080109395357380613
2019-01-01 15:32:30,“General” McChrystal got fired like a dog by O...,01-01-2019 15:32:30,69306,False,1080124615920373760


>None of the rows in is_retweet is 'True' so that column and 'id_str' can be dropped. Also, the donation dataset is only granular down to the day. I will group all tweets from one day into a single string. The favorite_count will also be summed per day. 

In [9]:
df_tweets.drop(columns=['created_at','is_retweet','id_str'],inplace = True)

In [10]:
df_tweets.head()

,text,favorite_count
Date,,
2019-01-01 12:51:34,Dr. Sebastian Gorka a very good and talented g...,76622
2019-01-01 13:08:29,HAPPY NEW YEAR TO EVERYONE INCLUDING THE HATER...,256114
2019-01-01 14:25:32,Happy New Year!,277623
2019-01-01 14:32:01,The Democrats much as I suspected have allocat...,101836
2019-01-01 15:32:30,“General” McChrystal got fired like a dog by O...,69306


<a id='day_tweet'></a>

In [11]:
count_df= df_tweets.resample('D')[['text']].count()

In [12]:
count_df.columns= ['tweet_count']

In [13]:
#Join all the tweets for each day.
df_text= df_tweets.resample('D').agg(' '.join)

In [14]:
#Sum all favorite counts perday for all tweets.
df_fav= df_tweets.resample('D').sum()

#Merge the aggregated daily tweets and favorite counts into one dataset.
tweet_df= df_text.merge(df_fav,left_index=True,right_index=True)

In [15]:
tweet_df.head()

,text,favorite_count
Date,,
2019-01-01,Dr. Sebastian Gorka a very good and talented g...,1827632
2019-01-02,For FAR TOO LONG Senate Democrats have been Ob...,431747
2019-01-03,Important meeting today on Border Security wit...,1091820
2019-01-04,Michael Pillsbury interviewed by @cvpayne: “Th...,942401
2019-01-05,Great new book by Dr. Robert Jeffress “Choosin...,1247133


In [16]:
tweet_df= tweet_df.merge(count_df,left_index=True,right_index=True)

### Confirm daily tweets have been aggregated daily.

In [17]:
tweet_df.loc['2019-01-01']['text']

'Dr. Sebastian Gorka a very good and talented guy has a great new book just out “Why We Fight.” Lots of insight - Enjoy! HAPPY NEW YEAR TO EVERYONE INCLUDING THE HATERS AND THE FAKE NEWS MEDIA! 2019 WILL BE A FANTASTIC YEAR FOR THOSE NOT SUFFERING FROM TRUMP DERANGEMENT SYNDROME. JUST CALM DOWN AND ENJOY THE RIDE GREAT THINGS ARE HAPPENING FOR OUR COUNTRY! Happy New Year! The Democrats much as I suspected have allocated no money for a new Wall. So imaginative! The problem is without a Wall there can be no real Border Security - and our Country must finally have a Strong and Secure Southern Border! “General” McChrystal got fired like a dog by Obama. Last assignment a total bust. Known for big dumb mouth. Hillary lover! https://t.co/RzOkeHl3KV One thing has now been proven. The Democrats do not care about Open Borders and all of the crime and drugs that Open Borders bring! Congratulations to President @JairBolsonaro who just made a great inauguration speech - the U.S.A. is with you! Bord

<a id='remove'></a>
>There is some html in the tweets that might just add noise to my model. Below I remove it. 

In [18]:
#Remove html from tweet text.
tweet_df['text']= tweet_df['text'].apply(lambda text:re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text))


In [19]:
tweet_df['text'][0]

'Dr. Sebastian Gorka a very good and talented guy has a great new book just out “Why We Fight.” Lots of insight - Enjoy! HAPPY NEW YEAR TO EVERYONE INCLUDING THE HATERS AND THE FAKE NEWS MEDIA! 2019 WILL BE A FANTASTIC YEAR FOR THOSE NOT SUFFERING FROM TRUMP DERANGEMENT SYNDROME. JUST CALM DOWN AND ENJOY THE RIDE GREAT THINGS ARE HAPPENING FOR OUR COUNTRY! Happy New Year! The Democrats much as I suspected have allocated no money for a new Wall. So imaginative! The problem is without a Wall there can be no real Border Security - and our Country must finally have a Strong and Secure Southern Border! “General” McChrystal got fired like a dog by Obama. Last assignment a total bust. Known for big dumb mouth. Hillary lover!  One thing has now been proven. The Democrats do not care about Open Borders and all of the crime and drugs that Open Borders bring! Congratulations to President @JairBolsonaro who just made a great inauguration speech - the U.S.A. is with you! Border Security and the Wal

> HTML has been removed.

In [20]:
#Remove all emojis from tweet texts.
rgx = re.compile('[^' + ''.join(string.printable) + ']')
tweet_df['text']= tweet_df['text'].apply(lambda text:rgx.sub('', text))


<a id='campaign'></a>
## Import Trump campaign donation dataset

> The donations dataset was gathered from the Federal Elections Commission(FEC) website found [here](https://www.fec.gov/data/receipts/individual-contributions/?committee_id=C00580100&two_year_transaction_period=2020&min_date=01%2F01%2F2019&max_date=12%2F31%2F2019). The full dataset includes all information about individual donations to Trump's campaign from Jan 1, 2019 to June 30, 2019. 

In [21]:
#Load in dataset of all Trump's donations.
df_donations= pd.read_csv('../datasets/schedule_a-2019-08-07T17_19_59.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5,11,28,29,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
df_donations.shape

(373615, 79)

In [23]:
df_donations.head()

,committee_id,committee_name,report_year,report_type,image_number,line_number,transaction_id,file_number,committee_name.1,entity_type,...,filing_form,link_id,is_individual,memo_text,two_year_transaction_period,schedule_type,increased_limit,sub_id,pdf_url,line_number_label
0,C00580100,"DONALD J. TRUMP FOR PRESIDENT, INC.",2019,Q1,201907089150471229,17A,SA17A.82963,1337251,NaN,IND,...,F3P,4071020191659100254,t,NaN,2020,SA,NaN,4071920191660307605,http://docquery.fec.gov/cgi-bin/fecimg/?201907...,Contributions From Individuals/Persons Other T...
1,C00580100,"DONALD J. TRUMP FOR PRESIDENT, INC.",2019,Q1,201907089150471229,17A,SA17A.91141,1337251,NaN,IND,...,F3P,4071020191659100254,t,NaN,2020,SA,NaN,4071920191660307606,http://docquery.fec.gov/cgi-bin/fecimg/?201907...,Contributions From Individuals/Persons Other T...
2,C00580100,"DONALD J. TRUMP FOR PRESIDENT, INC.",2019,Q1,201907089150471230,17A,SA17A.91142,1337251,NaN,IND,...,F3P,4071020191659100254,t,NaN,2020,SA,NaN,4071920191660307607,http://docquery.fec.gov/cgi-bin/fecimg/?201907...,Contributions From Individuals/Persons Other T...
3,C00580100,"DONALD J. TRUMP FOR PRESIDENT, INC.",2019,Q1,201907089150471230,17A,SA17A.91143,1337251,NaN,IND,...,F3P,4071020191659100254,t,NaN,2020,SA,NaN,4071920191660307608,http://docquery.fec.gov/cgi-bin/fecimg/?201907...,Contributions From Individuals/Persons Other T...
4,C00580100,"DONALD J. TRUMP FOR PRESIDENT, INC.",2019,Q1,201907089150471230,17A,SA17A.52998,1337251,NaN,IND,...,F3P,4071020191659100254,t,NaN,2020,SA,NaN,4071920191660307609,http://docquery.fec.gov/cgi-bin/fecimg/?201907...,Contributions From Individuals/Persons Other T...


In [24]:
df_donations.columns

Index(['committee_id', 'committee_name', 'report_year', 'report_type',
       'image_number', 'line_number', 'transaction_id', 'file_number',
       'committee_name.1', 'entity_type', 'entity_type_desc',
       'unused_contbr_id', 'contributor_prefix', 'contributor_name',
       'recipient_committee_type', 'recipient_committee_org_type',
       'recipient_committee_designation', 'contributor_first_name',
       'contributor_middle_name', 'contributor_last_name',
       'contributor_suffix', 'contributor_street_1', 'contributor_street_2',
       'contributor_city', 'contributor_state', 'contributor_zip',
       'contributor_employer', 'contributor_occupation', 'contributor_id',
       'receipt_type', 'receipt_type_desc', 'receipt_type_full', 'memo_code',
       'memo_code_full', 'contribution_receipt_date',
       'contribution_receipt_amount', 'contributor_aggregate_ytd',
       'candidate_id', 'candidate_name', 'candidate_first_name',
       'candidate_last_name', 'candidate_middle_na

In [25]:
df_donations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373615 entries, 0 to 373614
Data columns (total 79 columns):
committee_id                             373615 non-null object
committee_name                           373615 non-null object
report_year                              373615 non-null int64
report_type                              373615 non-null object
image_number                             373615 non-null int64
line_number                              373615 non-null object
transaction_id                           373615 non-null object
file_number                              373615 non-null int64
committee_name.1                         0 non-null float64
entity_type                              373615 non-null object
entity_type_desc                         373615 non-null object
unused_contbr_id                         4 non-null object
contributor_prefix                       126227 non-null object
contributor_name                         373615 non-null object
recip

<a id='camp_clean'></a>
> Change 'contribution_receipt_date to a datetime object and set as index. 

In [26]:
df_donations['Date']= pd.to_datetime(df_donations['contribution_receipt_date'])
df_donations.set_index('Date', inplace=True)
df_donations.sort_index(inplace=True,ascending=True)

>There is a lot of good information, provided in the dataset from employer of donor to their location. However for the purposes of this project the only information I care about is the total daily donations. 

In [27]:
daily_donations= pd.DataFrame(df_donations['contribution_receipt_amount'].resample('D').sum())

In [28]:
#Confirm the dataset aggregated correctly.
daily_donations.head()

,contribution_receipt_amount
Date,
2019-01-01,80596.00
2019-01-02,38655.24
2019-01-03,42904.48
2019-01-04,38082.68
2019-01-05,24696.23


<a id='combine'></a>
## Combine datasets into one called full_dataset.

In [29]:
df= tweet_df.merge(daily_donations,left_index=True,right_index=True)

In [30]:
df.head()

,text,favorite_count,tweet_count,contribution_receipt_amount
Date,,,,
2019-01-01,Dr. Sebastian Gorka a very good and talented g...,1827632,12,80596.00
2019-01-02,For FAR TOO LONG Senate Democrats have been Ob...,431747,3,38655.24
2019-01-03,Important meeting today on Border Security wit...,1091820,9,42904.48
2019-01-04,Michael Pillsbury interviewed by @cvpayne: The...,942401,7,38082.68
2019-01-05,Great new book by Dr. Robert Jeffress Choosing...,1247133,12,24696.23


<a id='remove'></a>
## Remove punctuation and convert all words to lowercase

In [31]:
df['text'] = df['text'].apply(remove_punctuation)

In [32]:
df['text']= df['text'].apply(lambda x: x.lower())

In [33]:
#One tweet text is saved as a float. Change to string.
df['text']=df['text'].astype(str)

<a id='export'></a>

In [34]:
df.to_csv('../datasets/full_dataset.csv')

<a id='conclusions'></a>
## Conclusion  
  
The data is now clean and ready for the exploratory data analysis part of the process. Overall, the data was pretty clean. Much of the cleaning process was removing parts of text that were going to just provide noise to the analysis and modeling processes. The observations also had to be aggregated into per day observations because for ease of use.  
Note that there was a lot of information provided in both original datasets that might have predictive signal, but were outside the scope of this project. It would be interesting to go back and look at what insights could be gained from the dropped information.